 <h1 style="background-color:#FD0E35
;font-family:newtimeroman;font-size:225%;text-align:center;border-radius: 15px 50px;"> 📈 EDA: Tesla Stock Price Financial Analysis & Visualization 📈</h1><a id=0></a>
 
 ### What is included:

 #  [1. Tesla Stock Price preliminary analysis](#1)
   [1.1 Imporint stock data with statistical summary](#1.1)
   
   [1.2 Stock data line graph](#1.2)
   
   [1.3 Implementation of the Rolling Window calculation](#1.3)
   
   [1.4 Summary of the part one](#1.4)
   
 #  [2. Building a trading strategy](#2)
 [2.1 Calculating the profit](#2.1)
 
 [2.2 Analyse of the income](#2.2)
   
 [2.3 Summary of the part two](#2.3)
 
  #  [3. Calculating the risk of price drop](#3)
 [3.1 Showing the histogram of the stock return](#3.1)
 
 [3.2 Calculating the probability that the price will drop 10% during a day](#3.2)
   
 [3.3 Calculating the probability that the price will drop 25% over a year](#3.3)
 
 [3.4 Calculate Value at risk (VaR)](#3.4)
 
 [3.5 Estimate the average stock return with 90% Confidence Interval](#3.4)
 
 [3.6 Summary of the part three](#3.5)

 #  [4. Building our trading model](#4)
 [4.1 Data Munging](#4.1)
  

![](https://ei.marketwatch.com/Multimedia/2017/07/05/Photos/NS/MW-FP681_teslas_20170705130102_NS.gif?uuid=866514ee-61a3-11e7-9945-9c8e992d421e)

In [ ]:
#This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import statsmodels.formula.api as smf
import requests
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
pd.options.mode.chained_assignment = None  # default='warn'
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
%matplotlib inline
import plotly.graph_objects as go
from plotly.subplots import make_subplots
palette = ['#F5DF4D', '#939597', '#0F4C81', '#FF6F61', '#5F4B8B', '#88B04B', '#92A8D1', '#F7CAC9', '#955251',\
           '#B163A3', '#009473']
palette.reverse()
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current sessione current session

 <h1 style="background-color:#FF9933
;font-family:newtimeroman;font-size:200%;text-align:center;border-radius: 15px 50px;"> 📈 Importing data, simple preliminary analysis
 📈</h1> <a id=1></a>

<h1 style="background-color:#FF9933
;font-family:newtimeroman;font-size:150%;text-align:left;border-radius: 15px 50px;"> 📈 Importing data:
 </h1> <a id=1.1></a>

In [ ]:
tesla = pd.read_csv('/kaggle/input/tesla-stock-data-from-2010-to-2020/TSLA.csv')
tesla.head()

<h1 style="background-color:#FF9933
;font-family:newtimeroman;font-size:150%;text-align:left;border-radius: 15px 50px;"> 📈 Summary statistics of a DataFrame:
 </h1> <a id=1.1></a>

In [ ]:
tesla.set_index('Date')


<h1 style="background-color:#FF9933
;font-family:newtimeroman;font-size:150%;text-align:left;border-radius: 15px 50px;"> 📈 Stock data line graph:
 </h1> <a id=1.2></a>

In [ ]:
plt.figure(figsize=(10, 8))
tesla['Close'].plot()
plt.show()

<h1 style="background-color:#FF9933
;font-family:newtimeroman;font-size:150%;text-align:left;border-radius: 15px 50px;"> 📈 Creation of the new column "Price difference"
 </h1> <a id=1.1></a>

In [ ]:
tesla['PriceDiff'] = tesla['Close'].shift(-1) - tesla['Close']
print(tesla['PriceDiff'])

<h1 style="background-color:#FF9933
;font-family:newtimeroman;font-size:150%;text-align:left;border-radius: 15px 50px;"> 📈 Creation of the new column "Daily return":
 </h1> <a id=1.1></a>

In [ ]:
tesla['Return'] = tesla['PriceDiff'] /tesla['Close']
print(tesla['Return'])

<h1 style="background-color:#FF9933
;font-family:newtimeroman;font-size:150%;text-align:left;border-radius: 15px 50px;"> 📈 Implementation of the Rolling Window calculation:
 </h1> <a id=1.3></a>

#### First time using Moving average, a little description for beginners as me 
In statistics, a moving average (rolling average or running average) is a calculation to analyze data points by creating a series of averages of different subsets of the full data set. It is also called a moving mean (MM). or rolling mean and is a type of finite impulse response filter. Variations include: simple, cumulative, or weighted forms (described below). Highly used for financial analyses.


In [ ]:
tesla['ma50'] = tesla['Close'].rolling(50).mean()
#plot the moving average
plt.figure(figsize=(10, 8))
tesla['ma50'].plot(label='MA50')
tesla['Close'].plot(label='Close')
plt.legend()
plt.show()

<h1 style="background-color:#FF9933
;font-family:newtimeroman;font-size:150%;text-align:left;border-radius: 15px 50px;"> 📈 Summary of the preliminary analisys:
 </h1> <a id=1.4></a>

#### The above is a ten years chart of Tesla Stock with simple moving averages. The blue line is the shorter, 50-day moving average. Most traders will use the cross of the short-term moving average above the longer-term moving average to initiate a long position and identify the start of a bullish trend. As we can see there is a growing trend.

 <h1 style="background-color:#FF9933
;font-family:newtimeroman;font-size:200%;text-align:center;border-radius: 15px 50px;"> 📈 Building a trading strategy
 📈</h1> <a id=2></a>

<h1 style="background-color:#FF9933
;font-family:newtimeroman;font-size:150%;text-align:left;border-radius: 15px 50px;"> 📈 Adding new columns:
 </h1> <a id=2.1></a>

In [ ]:
tesla['MA10'] = tesla['Close'].rolling(10).mean()
tesla['MA50'] = tesla['Close'].rolling(50).mean()
tesla = tesla.dropna()
tesla.head()

In [ ]:
tesla['Shares'] = [1 if tesla.loc[ei, 'MA10']>tesla.loc[ei, 'MA50'] else 0 for ei in tesla.index]

<h1 style="background-color:#FF9933
;font-family:newtimeroman;font-size:150%;text-align:left;border-radius: 15px 50px;"> 📈 Calculating the profit:
 </h1> <a id=2.1></a>

#### The profit is calculated as the close price of (tomorrow - the close price of today) Otherwise the profit is 0.

In [ ]:
tesla['Close1'] = tesla['Close'].shift(-1)
tesla['Profit'] = [tesla.loc[ei, 'Close1'] - tesla.loc[ei, 'Close'] if tesla.loc[ei, 'Shares']==1 else 0 for ei in tesla.index]
tesla['Profit'].plot()
plt.axhline(y=0, color='red')

<h1 style="background-color:#FF9933
;font-family:newtimeroman;font-size:150%;text-align:left;border-radius: 15px 50px;"> 📈 What would happen if we keep our investment:
 </h1> <a id=2.2></a>

In [ ]:
tesla['wealth'] = tesla['Profit'].cumsum()
tesla.tail()

In [ ]:
tesla['wealth'].plot()
plt.title('Total money you have made: {}'.format(tesla.loc[tesla.index[-2], 'wealth']))

<h1 style="background-color:#FF9933
;font-family:newtimeroman;font-size:150%;text-align:left;border-radius: 15px 50px;"> 📈 Summary of the preliminary analisys:
 </h1> <a id=1.4></a>

#### As we can see our strategy gave as the possibility to made good money during all of these years, if you would like to implement it for other stock prices feel free to use it and create your own simple trading strategy by copying the codes above.

 <h1 style="background-color:#FF9933
;font-family:newtimeroman;font-size:200%;text-align:center;border-radius: 15px 50px;"> 📈 Calculating the risk of price drop
 📈</h1> <a id=3></a>

<h1 style="background-color:#FF9933
;font-family:newtimeroman;font-size:150%;text-align:left;border-radius: 15px 50px;"> 📈 Adding column regarding the return:
 </h1> 

In [ ]:
tesla['LogReturn'] = np.log(tesla['Close']).shift(-1) - np.log(tesla['Close'])
print(tesla['LogReturn'])

<h1 style="background-color:#FF9933
;font-family:newtimeroman;font-size:150%;text-align:left;border-radius: 15px 50px;"> 📈 Showing the histogram of log return of Tesla's stock:
    <a id=3.1></a>

In [ ]:
from scipy.stats import norm
mu = tesla['LogReturn'].mean()
sigma = tesla['LogReturn'].std(ddof=1)

density = pd.DataFrame()
density['x'] = np.arange(tesla['LogReturn'].min()-0.01, tesla['LogReturn'].max()+0.01, 0.001)
density['pdf'] = norm.pdf(density['x'], mu, sigma)

tesla['LogReturn'].hist(bins=50, figsize=(15, 8))
plt.plot(density['x'], density['pdf'], color='red')
plt.show()

<h1 style="background-color:#FF9933
;font-family:newtimeroman;font-size:150%;text-align:left;border-radius: 15px 50px;"> 📈 Calculating the probability of 10 percentage price drop in a day:
    <a id=3.2></a>

In [ ]:
prob_return1 = norm.cdf(-0.10, mu, sigma)
print('The probability of dropping over 10% in one day ', prob_return1)

#### The probability of dropping over 10% in one day  0.0006816601499662236

<h1 style="background-color:#FF9933
;font-family:newtimeroman;font-size:150%;text-align:left;border-radius: 15px 50px;"> 📈 Calculating the probability of 25 percentage price drop over a year:
    <a id=3.3></a>

In [ ]:
mu220 = 365*mu
sigma220 = (365**0.5) * sigma
drop20 = None
print('The probability of dropping over 25% over a year: ', drop20)

<h1 style="background-color:#FF9933
;font-family:newtimeroman;font-size:150%;text-align:left;border-radius: 15px 50px;"> 📈 Calculate Value at risk (VaR):
    <a id=3.4></a>

In [ ]:
VaR = norm.ppf(0.05, mu, sigma)
print('Single day value at risk is: ', VaR)

#### Single day value at risk is:  -0.050719143759555683


In [ ]:
# Quatile 
# 5% quantile
print('5% quantile ', norm.ppf(0.05, mu, sigma))
# 95% quantile
print('95% quantile ', norm.ppf(0.95, mu, sigma))

<h1 style="background-color:#FF9933
;font-family:newtimeroman;font-size:150%;text-align:left;border-radius: 15px 50px;"> 📈 Estimate the average stock return with 90% Confidence Interval:
    <a id=3.5></a>

In [ ]:
tesla['logReturn'] = np.log(tesla['Close'].shift(-1)) - np.log(tesla['Close'])
sample_size = tesla['logReturn'].shape[0]
sample_mean = tesla['logReturn'].mean()
sample_std = tesla['logReturn'].std(ddof=1) / sample_size**0.5
# left and right quantile
z_left = norm.ppf(0.05)
z_right = norm.ppf(0.95)

# upper and lower bound
interval_left = sample_mean+z_left*sample_std
interval_right = sample_mean+z_right*sample_std

In [ ]:
print('90% confidence interval is ', (interval_left, interval_right))

#### 90% confidence interval is  (0.00024576826523648224, 0.0024087428049162068)


<h1 style="background-color:#FF9933
;font-family:newtimeroman;font-size:150%;text-align:left;border-radius: 15px 50px;"> 📈 Summary of the part three:
    <a id=3.5></a>

#### Our simple calculation and predictions show that is nearly impossible that a price would drop during a day over 10% or during a year over 25%. The amount of calculated value at risk and what we got from the confidence interval shows us that Tesla is a super safe investment. Of course it is only a simple prediction and analyse that can not be taken serious but I hop that this example can show others the path that begginers can start from.



![](https://media.tenor.com/images/99d431d9b3db1b10121cb7c90523c05a/tenor.gif)

 <h1 style="background-color:#FF9933
;font-family:newtimeroman;font-size:200%;text-align:center;border-radius: 15px 50px;"> 📈  Building our trading model 
 📈</h1> <a id=3></a>

<h1 style="background-color:#FF9933
;font-family:newtimeroman;font-size:150%;text-align:left;border-radius: 15px 50px;"> 📈 Data Spliting:
    <a id=4.1></a>

In [ ]:
tesla.head

In [ ]:
Train = tesla.iloc[-2000:-1000, :]
Test = tesla.iloc[-1000:, :]
print(Train.shape, Test.shape)

In [ ]:
from pandas.plotting import scatter_matrix
sm = scatter_matrix(Train, figsize=(10, 10))

 <h1 style="background-color:#FD0E35
;font-family:newtimeroman;font-size:225%;text-align:center;border-radius: 15px 50px;"> 📈 Thank you all for watching please leave feedback, It would really help me to develop new skills and learn new things 😃 📈</h1><a id=0></a>